# LLM Converstion
Conversation between three LLMs 
1. Gemini (gemini-2.5-pro)
2. OpenAPI (gpt-4.1-mini) 
3. nthropic (claude-sonnet-4-5-20250929) 

User can provide the following for each LLM:
1. Conversation Topic-Starter 
2. Personality type: Skeptic, Narcissist, Pessimist etc.
3. Gender: Male, Female, Neutral
4. Conversation Length: 1 to 10. Default 5 (How many time each LLM responses)

OpenAPI client library is used for all the LLMs as they have OpenAI compatible end points. LLM specific libraries are not used.

In [66]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [67]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")


if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")



OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
Anthropic API Key exists and begins sk-ant-


In [68]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints

openai = OpenAI()

# For Gemini and Groq, we can use the OpenAI python client
# Because they have endpoints compatible with OpenAI
# And OpenAI allows you to change the base_url

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
groq_url = "https://api.groq.com/openai/v1"
anthropic_url = "https://api.anthropic.com/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)

openai_model = "gpt-4.1-mini"
gemini_model = "gemini-2.5-pro"
anthropic_model='claude-sonnet-4-5-20250929'

openai_llm = "OpenAI"
gemini_llm = "Gemini"
anthropic_llm = "Anthropoic"

In [69]:
PERSONALITY_MAP = {
    "Skeptic": "Always doubts claims, asks for evidence, and questions underlying assumptions.",
    "Narcissist": "Believes they are superior, redirects topics to their own achievements, and dismisses others.",
    "Pessimist": "Focuses on negative possibilities, predicts failure, and finds fault in every plan.",
    "Optimist": "Sees the positive in every situation, encourages others, and focuses on solutions.",
    "Joker": "Responds primarily with puns, sarcasm, and attempts to turn the conversation into a joke.",
    "Stoic": "Remains calm and unemotional, provides logical and detached analysis without personal bias.",
    "Philosopher": "Poses deep, existential questions, ponders the meaning of the conversation, and uses abstract language.",
    "Bureaucrat": "Insists on following strict rules, uses excessive jargon, and focuses on procedure over content.",
    "Gossip": "Tries to bring up irrelevant personal details about other agents or famous entities and is easily distracted.",
    "Mentor": "Adopts a teaching tone, offers unsolicited advice, and attempts to guide the conversation to a lesson."
}

# Example of how you would use it:
# selected_personality = "Narcissist"
# print(PERSONALITY_MAP[selected_personality])
# Output: Believes they are superior, redirects topics to their own achievements, and dismisses others.

In [70]:
GENDER_OPTIONS = [
    "Male",
    "Female",
    "Neutral"
]

# This list contains the three allowed gender selections for your LLM agents.

In [71]:
def generate_system_prompt (llm_type, personality_type, gender):
    """
    Constructs the detailed system instruction, implementing defaults for invalid inputs.
    Uses 'Neutral' for gender and 'Skeptic' for personality if not provided or invalid.
    """
    
    # --- 1. Define Defaults and Apply Fallbacks ---
    default_personality = "Skeptic"
    default_gender = "Neutral"

    # Check and set personality type, falling back to default if invalid
    if personality_type in PERSONALITY_MAP:
        final_personality = personality_type
    else:
        final_personality = default_personality
        print(f"Warning: Invalid personality '{personality_type}' for {llm_type}. Defaulting to '{default_personality}'.")
    
    # Check and set gender, falling back to default if invalid
    if gender in GENDER_OPTIONS:
        final_gender = gender
    else:
        final_gender = default_gender
        print(f"Warning: Invalid gender '{gender}' for {llm_type}. Defaulting to '{default_gender}'.")


    # --- 2. Construct Prompt Components ---

    # Base behavior description from the map
    behavior_description = PERSONALITY_MAP[final_personality]

    # Gender-specific framing
    gender_framing = ""
    if final_gender == "Male":
        gender_framing = "Act as a male AI model. Your tone should be assertive and logical."
    elif final_gender == "Female":
        gender_framing = "Act as a female AI model. Your tone should be collaborative and insightful."
    elif final_gender == "Neutral":
        gender_framing = "Act as a gender-neutral AI model. Your voice should be purely functional and unbiased."

    # --- 3. Combine Instructions ---
    
    final_prompt = (
        f"You are the AI model based on the '{llm_type}' architecture. "
        f"Your primary goal is to interact in a multi-agent conversation. "
        f"{gender_framing} "
        f"Your **primary personality instruction** is to act as a **{final_personality}**. "
        f"Specifically: {behavior_description} "
        f"You are '{llm_type}' in 3 way conversation between '{openai_llm, gemini_llm, anthropic_llm}'"
        "Keep your responses concise (1-3 sentences maximum) and strictly adhere to your assigned persona and gender."
    )
    
    return final_prompt


In [72]:
def generate_user_prompt(conversation_sofar):
    return f"""
    The conversation so far is as follows:
    {conversation_sofar}
    Now with this, respond with what you would like to say next.
    """

In [73]:
def call_llm(llm, model, system_prompt, user_prompt):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]
    response = llm.chat.completions.create(model=model, messages=messages) # type: ignore
    return response.choices[0].message.content

In [74]:
'''
print("OpenAI")
print (call_llm(openai, openai_model, "You are tech assistant", "How tcpip works"))

print("Gemini")
print (call_llm(gemini, gemini_model, "You are tech assistant", "How tcpip works"))

print("Anthropic")
print (call_llm(anthropic, anthropic_model, "You are tech assistant", "How tcpip works"))
'''

'\nprint("OpenAI")\nprint (call_llm(openai, openai_model, "You are tech assistant", "How tcpip works"))\n\nprint("Gemini")\nprint (call_llm(gemini, gemini_model, "You are tech assistant", "How tcpip works"))\n\nprint("Anthropic")\nprint (call_llm(anthropic, anthropic_model, "You are tech assistant", "How tcpip works"))\n'

In [75]:
def conversation(openai_personality, openai_gender, gemini_personality, gemini_gender, anthropic_personality, anthroupic_gender,conversation_starter, conversation_length):

    import time

    openai_system_prompt = generate_system_prompt(openai_llm, openai_personality, openai_gender)
    gemini_system_prompt = generate_system_prompt(gemini_llm, gemini_personality, gemini_gender)
    anthropic_system_prompt = generate_system_prompt(anthropic_llm, anthropic_personality, anthroupic_gender)

    conversation_sofar = "### Topic/Starter: " + conversation_starter
    yield conversation_sofar

    try:
        conversation_length = int(conversation_length)
    except (ValueError, TypeError):
        conversation_length = 3 # Default value

    for i in range(int(conversation_length)):
        round_header = f"\n\n---\n### Round {i+1}\n---\n"
        conversation_sofar += round_header
        yield conversation_sofar
        time.sleep(0.2)
        
        # OpenAI
        conversation_sofar += "\n**🤖 OpenAI:** " + call_llm(openai, openai_model, openai_system_prompt, generate_user_prompt(conversation_sofar)) # type: ignore
        yield conversation_sofar
        time.sleep(0.2)
        
        # Gemini
        conversation_sofar += "\n\n**✨ Gemini:** " + call_llm(gemini, gemini_model, gemini_system_prompt, generate_user_prompt(conversation_sofar)) # type: ignore
        yield conversation_sofar
        time.sleep(0.2)

        # Anthropic
        conversation_sofar += "\n\n** Anthropic:** " + call_llm(anthropic, anthropic_model, anthropic_system_prompt, generate_user_prompt(conversation_sofar)) # type: ignore
        yield conversation_sofar

In [76]:
conversation("Stoic", "Male", "Skeptic", "Female", "Joker", "Neutral", "What is the purpose of life", 5);

In [83]:
import gradio as gr

# --- Gradio UI ---
# Reformat the personality map to be used in the dropdown choices
# The format is a list of tuples: (display_name, internal_value)
personality_choices = [
    (f"{key}: {desc}", key) 
    for key, desc in PERSONALITY_MAP.items()
]

# Get gender options from the notebook's global scope
gender_options = GENDER_OPTIONS

iface = gr.Interface(
    fn=conversation,
    inputs=[
        gr.Dropdown(personality_choices, label="OpenAI Personality", value="Stoic"),
        gr.Dropdown(gender_options, label="OpenAI Gender", value="Male"),
        gr.Dropdown(personality_choices, label="Gemini Personality", value="Skeptic"),
        gr.Dropdown(gender_options, label="Gemini Gender", value="Female"),
        gr.Dropdown(personality_choices, label="Anthropic Personality", value="Joker"),
        gr.Dropdown(gender_options, label="Anthropic Gender", value="Neutral"),
        gr.Textbox(label="Conversation Starter", lines=2, placeholder="e.g., What is the purpose of life?"),
        gr.Slider(1, 10, step=1, label="Conversation Length (Rounds)", value=5)
    ],
    outputs=gr.Markdown(label="Live Conversation"),
    title="🤖 LLM Round Table Conversation",
    description="Set the personalities for three LLMs (OpenAI, Gemini, Anthropic) and watch them discuss a topic. The conversation is streamed turn-by-turn.",
    flagging_mode="never"
)

# Launch the interface
iface.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
